In [ ]:
# 🧩 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os, zipfile, shutil, time
from IPython.display import display
import ipywidgets as widgets

# =========================================================
# 📂 STEP 1: Interactive folder navigation
# =========================================================

def list_drive_folders(path="/content/drive/My Drive"):
    """List subfolders in a given Drive path."""
    try:
        folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
        return folders
    except Exception as e:
        print(f"⚠️ Cannot access: {path} — {e}")
        return []

def select_folder(start_path="/content/drive/My Drive"):
    """Interactive folder selector widget."""
    current_path = start_path

    while True:
        folders = list_drive_folders(current_path)
        print(f"\n📁 Current path: {current_path}")
        if not folders:
            print("No subfolders found here.")
        else:
            for i, f in enumerate(folders):
                print(f"[{i}] {f}")

        print("\n✅ Type a folder number to go inside, '..' to go up, or 'ok' to select this folder.")
        choice = input("Your choice: ").strip()

        if choice.lower() == "ok":
            return current_path
        elif choice == "..":
            # Move one level up
            if current_path != "/content/drive/My Drive":
                current_path = os.path.dirname(current_path)
            else:
                print("Already at top level.")
        elif choice.isdigit() and int(choice) < len(folders):
            current_path = os.path.join(current_path, folders[int(choice)])
        else:
            print("❌ Invalid choice, try again.")

# =========================================================
# 📦 STEP 2: Folder selection
# =========================================================
print("🔍 Navigate to the folder containing your ZIP files:")
zip_dir = select_folder()
print(f"\n✅ Selected ZIP folder: {zip_dir}")

# Create output + log paths in same parent directory
base_dir = os.path.dirname(zip_dir)
output_dir = os.path.join(base_dir, "unzipped")
log_path = os.path.join(base_dir, "unzipped_log.txt")
os.makedirs(output_dir, exist_ok=True)
print(f"📦 Output folder: {output_dir}")

# =========================================================
# ⚙️ STEP 3: Safe extraction logic
# =========================================================

def safe_move(src, dst_dir):
    base, ext = os.path.splitext(os.path.basename(src))
    dst = os.path.join(dst_dir, base + ext)
    counter = 1
    while os.path.exists(dst):
        dst = os.path.join(dst_dir, f"{base}_{counter}{ext}")
        counter += 1
    shutil.move(src, dst)
    return dst

# Load processed log if exists
processed = set()
if os.path.exists(log_path):
    with open(log_path, "r") as log:
        processed = {line.strip() for line in log.readlines()}
    print(f"📝 Loaded {len(processed)} previously processed zips.")

# Get ZIP list
zip_files = [f for f in os.listdir(zip_dir) if f.lower().endswith(".zip")]
print(f"Found {len(zip_files)} zip file(s).")

# =========================================================
# 🧩 STEP 4: Extraction loop
# =========================================================

for idx, zip_name in enumerate(zip_files, 1):
    if zip_name in processed:
        print(f"⏭️ Skipping {zip_name} (already done)")
        continue

    zip_path = os.path.join(zip_dir, zip_name)
    local_zip = f"/content/temp_{zip_name}"
    temp_extract = "/content/unzipped_temp"
    os.makedirs(temp_extract, exist_ok=True)

    print(f"\n🗜️ [{idx}/{len(zip_files)}] Extracting {zip_name}...")

    try:
        # Copy ZIP to local storage
        shutil.copy2(zip_path, local_zip)
        with zipfile.ZipFile(local_zip, "r") as zip_ref:
            zip_ref.extractall(temp_extract)

        extracted_files = os.listdir(temp_extract)
        if not extracted_files:
            print(f"⚠️ {zip_name} extracted 0 files — keeping ZIP.")
            continue

        success_count = 0
        for f in extracted_files:
            try:
                src = os.path.join(temp_extract, f)
                safe_move(src, output_dir)
                success_count += 1
            except Exception as e:
                print(f"⚠️ Couldn’t move {f}: {e}")

        # Verify and delete
        if success_count == len(extracted_files):
            os.remove(zip_path)
            print(f"✅ Verified & deleted {zip_name} ({success_count} files)")
            with open(log_path, "a") as log:
                log.write(f"{zip_name}\n")
        else:
            print(f"❌ Partial copy, keeping {zip_name}")

    except Exception as e:
        print(f"❌ Error with {zip_name}: {e}")

    finally:
        # Cleanup
        if os.path.exists(local_zip):
            os.remove(local_zip)
        shutil.rmtree(temp_extract, ignore_errors=True)

        # Show free space
        !df -h /content | tail -1
        time.sleep(10)

print("\n🎉 All ZIPs processed! Logs saved to:", log_path)
